In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from datetime import datetime
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score,r2_score,mean_squared_error,confusion_matrix
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor,VotingRegressor
from sklearn import preprocessing
from sklearn import utils
from xgboost import XGBRegressor

In [2]:
df=pd.read_csv(r'C:\Users\91948\CondaIBM\Final Project\Data Colletion\Wind turbine.csv')
df

,Date/Time,LV ActivePower (kW),Wind Speed (m/s),Theoretical_Power_Curve (KWh),Wind Direction (°)
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286
...,...,...,...,...,...
50525,31 12 2018 23:10,2963.980957,11.404030,3397.190793,80.502724
50526,31 12 2018 23:20,1684.353027,7.332648,1173.055771,84.062599
50527,31 12 2018 23:30,2201.106934,8.435358,1788.284755,84.742500
50528,31 12 2018 23:40,2515.694092,9.421366,2418.382503,84.297913


In [3]:
df.shape

(50530, 5)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50530 entries, 0 to 50529
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Date/Time                      50530 non-null  object 
 1   LV ActivePower (kW)            50530 non-null  float64
 2   Wind Speed (m/s)               50530 non-null  float64
 3   Theoretical_Power_Curve (KWh)  50530 non-null  float64
 4   Wind Direction (°)             50530 non-null  float64
dtypes: float64(4), object(1)
memory usage: 1.9+ MB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
LV ActivePower (kW),50530.0,1307.684332,1312.459242,-2.471405,50.677890,825.838074,2482.507568,3618.732910
Wind Speed (m/s),50530.0,7.557952,4.227166,0.000000,4.201395,7.104594,10.300020,25.206011
Theoretical_Power_Curve (KWh),50530.0,1492.175463,1368.018238,0.000000,161.328167,1063.776283,2964.972462,3600.000000
Wind Direction (°),50530.0,123.687559,93.443736,0.000000,49.315437,73.712978,201.696720,359.997589


In [6]:
df.isna().any

<bound method NDFrame._add_numeric_operations.<locals>.any of        Date/Time  LV ActivePower (kW)  Wind Speed (m/s)  \
0          False                False             False   
1          False                False             False   
2          False                False             False   
3          False                False             False   
4          False                False             False   
...          ...                  ...               ...   
50525      False                False             False   
50526      False                False             False   
50527      False                False             False   
50528      False                False             False   
50529      False                False             False   

       Theoretical_Power_Curve (KWh)  Wind Direction (°)  
0                              False               False  
1                              False               False  
2                              False               F

--Splitting the Data

In [7]:
x=df[['Wind Speed (m/s)','Wind Direction (°)']]
y = df['LV ActivePower (kW)']

In [8]:
x.head

<bound method NDFrame.head of        Wind Speed (m/s)  Wind Direction (°)
0              5.311336          259.994904
1              5.672167          268.641113
2              5.216037          272.564789
3              5.659674          271.258087
4              5.577941          265.674286
...                 ...                 ...
50525         11.404030           80.502724
50526          7.332648           84.062599
50527          8.435358           84.742500
50528          9.421366           84.297913
50529          9.979332           82.274620

[50530 rows x 2 columns]>

In [9]:
y.head

<bound method NDFrame.head of 0         380.047791
1         453.769196
2         306.376587
3         419.645905
4         380.650696
            ...     
50525    2963.980957
50526    1684.353027
50527    2201.106934
50528    2515.694092
50529    2820.466064
Name: LV ActivePower (kW), Length: 50530, dtype: float64>

In [10]:
x_train, x_test,y_train, y_test = train_test_split(x,y ,random_state=104,test_size=0.25)

In [11]:
y_train

28504      49.185532
41746    3603.016113
29608     607.884216
27937     777.071228
44653    1607.969971
            ...     
31240    3599.718994
40664     142.976593
39078    2192.166992
49881       0.000000
8261     3603.206055
Name: LV ActivePower (kW), Length: 37897, dtype: float64

--Checking the Metrix

In [12]:
sm=SVR()
model_sm=sm.fit(x_train,y_train)
y_sm=model_sm.predict(x_test)
print('R2-svm',r2_score(y_test,y_sm))
print('RMSE-svm',np.sqrt(mean_squared_error(y_test,y_sm)))

R2-svm 0.003210488886992935
RMSE-svm 1302.12133034028


In [13]:
xgr=XGBRegressor()
model_xgr=xgr.fit(x_train,y_train)
y_xgr=model_xgr.predict(x_test)
print('R2-xgb',r2_score(y_test,y_xgr))
print('RMSE-xgb',np.sqrt(mean_squared_error(y_test,y_xgr)))

R2-xgb 0.9143962322036632
RMSE-xgb 381.5892783561758


In [14]:
lnr=LinearRegression()
model_lnr=lnr.fit(x_train,y_train)
y_lnr=model_lnr.predict(x_test)
print('R2-lr',r2_score(y_test,y_lnr))
print('RMSE-lr',np.sqrt(mean_squared_error(y_test,y_lnr)))

R2-lr 0.8320590517273709
RMSE-lr 534.4757730194342


In [15]:
rdfr=RandomForestRegressor()
model_rdfr=rdfr.fit(x_train,y_train)
y_rdfr=model_rdfr.predict(x_test)
print('R2-rf',r2_score(y_test,y_rdfr))
print('RMSE-rf',np.sqrt(mean_squared_error(y_test,y_rdfr)))

R2-rf 0.9060224494955879
RMSE-rf 399.8174860269534


In [16]:
dtr=DecisionTreeRegressor()
model_dtr=dtr.fit(x_train,y_train)
y_dtr=model_dtr.predict(x_test)
print('R2-dt',r2_score(y_test,y_dtr))
print('RMSE-dt',np.sqrt(mean_squared_error(y_test,y_dtr)))

R2-dt 0.8356433160601979
RMSE-dt 528.7415121144388


--Hyper parameter Tuning

In [17]:
params={
 "learning_rate"    : [0.05, 0.01,0.03,0.1, 0.15, 0.2] ,
 "n_estimators"     : [50, 100, 150, 200, 500, 800,1000,1500] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15,20,25],
 "min_child_weight" : [ 1, 3, 5, 7 ,10,15,20,25],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "subsample"        : [ 0.1, 0.2 , 0.3, 0.4,0.6,0.8,1 ],
 "reg_lambda"       : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ,0.6,0.8,1],
 "reg_alpha"        : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7,0.9 ],
 "colsample_bylevel" : [ 0.3, 0.4, 0.5 , 0.7,0.9 ]
}

In [18]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n The Time taken is : %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [19]:
random_search=RandomizedSearchCV(xgr,param_distributions=params,n_iter=10,n_jobs=-1,cv=5,verbose=3)
start_time = timer(None)


In [33]:
random_search.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          callbacks=None, colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=0, gpu_id=-1,
                                          grow_policy='depthwise',
                                          importance_type=None,
                                          interaction_constraints='',
                                          learning_rate=0.3000...
                                        'colsample_bytree': [0.3, 0.4, 0.5, 0.7,
                                                             0.9],
 

In [34]:
timer(start_time)


 The Time taken is : 0 hours 24 minutes and 10.17 seconds.


In [35]:
random_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=0.5, colsample_bynode=1, colsample_bytree=0.4,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.05, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=4, max_leaves=0, min_child_weight=25, missing=nan,
             monotone_constraints='()', n_estimators=200, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [36]:
xgr=XGBRegressor(colsample_bylevel=0.4, colsample_bytree=0.3, gamma=0.1,
             learning_rate=0.01, max_depth=6, min_child_weight=25,
             n_estimators=1500, reg_alpha=0.1, reg_lambda=0.8, subsample=0.6)

In [37]:
x=xgr.fit(x_train,y_train)

In [38]:
yval=x.predict(x_test)

In [39]:
r2_score(y_test,yval)

0.9118739462184973

In [40]:
xgr=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,colsample_bynode=1, colsample_bytree=0.3, gamma=0.2,importance_type='gain', learning_rate=0.03, max_delta_step=0,max_depth=8, min_child_weight=25, missing=None, n_estimators=800,n_jobs=1, nthread=None, objective='reg:linear', random_state=0,reg_alpha=0.2, reg_lambda=0.8, scale_pos_weight=1, seed=None,silent=None, subsample=0.1, verbosity=1)

In [41]:
sm=SVR(gamma='auto',C=50,epsilon=0.3)

In [42]:
rfr=RandomForestRegressor(n_estimators=500,max_depth=4)

In [43]:
lnr=LinearRegression()

In [44]:
model=VotingRegressor([('lnr',lnr), ('rfr',rfr),('sm', sm),('xgr',xgr)],weights=[1,1,2,3])

In [ ]:
Model=model.fit(x_train, y_train)